In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Importar o definir la clase RecommenderNet
class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size=50, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size

        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        x = dot_user_movie + user_bias + movie_bias
        return tf.nn.sigmoid(x)

# Cargar el modelo desde el archivo .keras
model = load_model("recommender_model.keras", custom_objects={"RecommenderNet": RecommenderNet})

In [3]:
# Reutilizar un ID de usuario existente dentro del rango original (0 a 942)
nuevo_usuario_id = int(input("Ingrese el numero de usuario para poder dar nuestras 10 recomendaciones: "))

Ingrese el numero de usuario para poder dar nuestras 10 recomendaciones:  1


In [4]:
# Importar los datos de las películas
movies = pd.read_csv("moviescl.csv")

# Definir el número total de películas basándote en el DataFrame
num_movies = movies['movie_id'].nunique()

# Simulación de preferencias del usuario, al ser minimas a comparacion de las del entreanimeto no afectara
peliculas_vistas = [1, 50, 100]  # Películas que el usuario ha visto y le han gustado (IDs de ejemplo)
calificaciones = [5, 4, 5]  # Calificaciones para las películas vistas

# Crear el dataset del nuevo usuario
nuevo_usuario_data = pd.DataFrame({
    'user_id': [nuevo_usuario_id] * len(peliculas_vistas),
    'movie_id': peliculas_vistas,
    'rating': calificaciones
})

# Realizar predicciones para todas las películas
peliculas_disponibles = np.array(range(num_movies))
usuario_input = np.array([nuevo_usuario_id] * len(peliculas_disponibles))

# Combinar usuario y película en una matriz con dos columnas
entrada_modelo = np.vstack((usuario_input, peliculas_disponibles)).T

# Predecir la afinidad del usuario por todas las películas
predicciones = model.predict(entrada_modelo)
nuevo_usuario_recomendaciones = pd.DataFrame({
    'movie_id': peliculas_disponibles,
    'prediccion': predicciones.flatten()
})

# Hacer un merge con el DataFrame de películas para obtener el título de cada película recomendada
top_recomendaciones = nuevo_usuario_recomendaciones.merge(movies, on='movie_id')
top_recomendaciones = top_recomendaciones.sort_values(by='prediccion', ascending=False).head(10)

# Mostrar las recomendaciones
print("Top 10 recomendaciones para el nuevo usuario:")
print(top_recomendaciones[['title', 'prediccion']])

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Top 10 recomendaciones para el nuevo usuario:
                                      title  prediccion
30                      Crimson Tide (1995)    0.782455
35                          Mad Love (1995)    0.776632
129                      Kansas City (1996)    0.774743
296                      Ulee's Gold (1997)    0.767442
4                            Copycat (1995)    0.761694
356  One Flew Over the Cuckoo's Nest (1975)    0.759506
173          Raiders of the Lost Ark (1981)    0.759015
199                     Shining, The (1980)    0.755017
0                          Toy Story (1995)    0.754030
57                         Quiz Show (1994)    0.746714
